In [1]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Embedding, GlobalMaxPooling1D, Dropout, Input
import matplotlib.pyplot as plt
from util import plot_history
import feature_builder
from sklearn.model_selection import GridSearchCV

Cargo datasets

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

Armo features con el feature_builder

In [3]:
#        encoding_type='one_hot',
# target_dimensions=5120,
#        text_type='tfidf',
#        clean_text=True
def prepare_for_cnn(df):
    processed = feature_builder.process_dataset(
        df,
        encoding_type='one_hot',
        target_dimensions=512,
        text_type='tfidf',
        clean_text=True
    )
    to_numpy = processed.to_numpy()
    to_numpy = to_numpy.reshape(processed.shape[0], processed.shape[1], 1)
    return processed, to_numpy

In [4]:
%%time
processed, X_train = prepare_for_cnn(train_df)
y_train = train_df['target'].values

Wall time: 1min 29s


In [5]:
processed

,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,...,dim_502,dim_503,dim_504,dim_505,dim_506,dim_507,dim_508,dim_509,dim_510,dim_511
0,32.475311,-2.144690,0.235073,-8.519305,6.594190,-6.424323,1.368275,-5.885777,-1.433935,0.196432,...,0.012574,0.010552,-0.007710,-0.000427,-0.027498,-0.010187,-0.033228,0.022420,0.023524,-0.011515
1,66.458625,-0.112845,-0.531661,-8.295580,3.577381,-1.720669,3.179970,-6.349487,1.233517,-0.254740,...,0.022584,0.003194,0.004936,-0.005233,0.003483,-0.002423,-0.003053,0.013383,0.028071,-0.007650
2,-31.737352,15.742098,-13.481771,-9.495079,5.540941,0.003274,4.087029,-8.177045,1.074142,0.255132,...,-0.005039,-0.020975,0.013070,-0.045118,-0.009580,0.007863,0.008233,0.004032,0.013610,0.039947
3,39.875717,-0.946336,-5.048949,-8.892658,-2.341142,-0.791188,2.305560,-7.661946,5.434321,0.021486,...,0.009380,0.005456,-0.020752,-0.002851,0.006204,0.004729,-0.012616,0.030198,-0.002040,0.003090
4,14.321503,-1.137494,-9.186638,-8.814102,4.428187,-4.160263,2.875922,-6.352974,-0.377421,0.119877,...,-0.009251,0.021514,-0.004322,0.013153,-0.009635,0.008479,0.017712,0.013646,-0.018855,-0.014263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,19.777552,-2.993427,-0.592385,-9.029177,-1.423421,-2.153466,2.758134,-7.522952,-0.039655,0.055885,...,-0.027359,0.015659,0.005488,-0.000862,-0.029802,0.019609,-0.009630,0.002608,-0.014200,0.005405
7609,-22.805414,-2.918664,-10.771923,-9.167686,4.281891,0.460687,4.952134,-7.889697,0.265700,0.255025,...,0.013649,0.016342,-0.033323,0.020598,-0.017449,0.008115,0.003835,-0.023603,0.000442,-0.018229
7610,35.669908,-4.786070,10.595461,-9.389385,-9.361353,-11.140263,2.854098,-2.865885,2.723528,0.320391,...,-0.008885,0.016763,0.016427,0.042056,0.016496,-0.018444,0.019027,-0.025919,-0.032306,-0.014374
7611,-33.932851,-2.899164,-14.156761,-9.337379,2.999756,4.052540,5.323198,-9.400615,2.145643,-0.134802,...,-0.020572,0.031393,0.022579,0.003192,0.007734,0.015934,0.000450,-0.043591,0.023405,-0.007303


Creo el modelo y corro la CNN

In [6]:
def create_model(filters, kernel_sizes, pool_sizes, units, cnn_layers, dense_layers):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    
    for i in range(cnn_layers):
        model.add(Conv1D(filters=filters[i], kernel_size=kernel_sizes[i], activation='relu'))
    model.add(MaxPooling1D(2))#pool_sizes[i]))

    model.add(Flatten())
    
    for i in range(dense_layers):
        model.add(Dense(units[i], activation='relu'))

    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(1e-4), metrics=['accuracy'])
    
    return model

In [7]:
filters=[32,32,64]
kernel_size=[3,3,3]
pool_sizes=[2,2,2]
units=[64]
cnn_layers=3
dense_layers=1

In [8]:
#acc = 0.8
filters=[128,128]
kernel_size=[3,3,3]
pool_sizes=[2,2,2]
units=[32, 32]
cnn_layers=3
dense_layers=1

In [9]:
#acc = 0.80 cleaner
filters=[128,128,128]
kernel_size=[3,4,5]
pool_sizes=[2,2,2]
units=[128*3]
cnn_layers=3
dense_layers=1

In [ ]:
model = create_model(
    filters=filters,
    kernel_sizes=kernel_size,
    pool_sizes=pool_sizes,
    units=units,
    cnn_layers=cnn_layers,
    dense_layers=dense_layers
)
history = model.fit(X_train, y_train, epochs=20, shuffle=False, validation_split=0.2, batch_size=16)
plot_history(history)

Epoch 1/20
381/381 [==============================] - 46s 120ms/step - loss: 0.6617 - accuracy: 0.5989 - val_loss: 0.6211 - val_accuracy: 0.6684
Epoch 2/20
381/381 [==============================] - 45s 119ms/step - loss: 0.6283 - accuracy: 0.6588 - val_loss: 0.5778 - val_accuracy: 0.7249
Epoch 3/20
381/381 [==============================] - 45s 118ms/step - loss: 0.6018 - accuracy: 0.6910 - val_loss: 0.5580 - val_accuracy: 0.7452
Epoch 4/20
381/381 [==============================] - 45s 118ms/step - loss: 0.5588 - accuracy: 0.7223 - val_loss: 0.5495 - val_accuracy: 0.7564
Epoch 5/20
381/381 [==============================] - 45s 119ms/step - loss: 0.5012 - accuracy: 0.7580 - val_loss: 0.5253 - val_accuracy: 0.7702
Epoch 6/20
381/381 [==============================] - 46s 122ms/step - loss: 0.4450 - accuracy: 0.7989 - val_loss: 0.5235 - val_accuracy: 0.7741
Epoch 7/20
381/381 [==============================] - 45s 119ms/step - loss: 0.4091 - accuracy: 0.8141 - val_loss: 0.5088 - val_ac

Calculemos el test para kaggle

In [ ]:
ids = test_df['id']

processed, final_test = prepare_for_cnn(test_df)
prediction = model.predict_classes(final_test)

final_df = pd.DataFrame({'target': [x[0] for x in prediction]}, index=ids)
final_df

In [ ]:
final_df['target'].value_counts()

In [21]:
final_df.count()

target    3263
dtype: int64

In [22]:
final_df.to_csv('cnn.csv')

In [ ]:
model = KerasClassifier(build_fn=create_model, verbose=False, epochs=25)

filters = [32,64,128,1024,10000]
kernel_size = [2,3,5]
units = [64,128]

param_grid = dict(filters=filters,kernel_size=kernel_size,units=units)# 
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

#Run grid search
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
filters=[32,32,64]
kernel_size=[3,3,3]
pool_sizes=[2,2,2]
units=[[64], [64, 64], [64, 32]]
cnn_layers=[1,2,3,4]
dense_layers=[1,2]